In [ ]:


import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from tqdm import tqdm
import os


In [3]:

model = YOLO("best.pt")  


In [11]:
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Get video details
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Output writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_reid.mp4', fourcc, fps, (width, height))

# Initialize DeepSort
tracker = DeepSort(max_age=30, n_init=2) 

In [13]:
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    print(f"[INFO] Processing frame: {frame_idx}")
    
    # Run YOLOv8 inference
    results = model(frame)[0]  # [0] to get first result

    detections = []
    if results.boxes is not None and len(results.boxes) > 0:
        for i in range(len(results.boxes)):
            box = results.boxes[i]
            cls = int(box.cls.item())
            conf = float(box.conf.item())
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            if cls == 0 and conf > 0.3:  # Only class 0 (player)
                w, h = x2 - x1, y2 - y1
                detections.append(([x1, y1, w, h], conf, 'player'))

    # Update Deep SORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracked bounding boxes with IDs
    for track in tracks:
        if not track.is_confirmed() or track.track_id is None:
            continue

        x1, y1, x2, y2 = map(int, track.to_ltrb())
        track_id = track.track_id

        # Draw rectangle and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Write the output frame
    out.write(frame)

    # Optional: Show live preview (can comment out)
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_idx += 1

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("[DONE] Output saved to: output_reid.mp4")

[INFO] Processing frame: 0

0: 384x640 18 players, 2 referees, 505.7ms
Speed: 2.8ms preprocess, 505.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 1

0: 384x640 1 ball, 16 players, 2 referees, 410.5ms
Speed: 1.5ms preprocess, 410.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 2

0: 384x640 1 ball, 14 players, 2 referees, 357.7ms
Speed: 1.4ms preprocess, 357.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 3

0: 384x640 1 ball, 14 players, 2 referees, 368.7ms
Speed: 1.5ms preprocess, 368.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 4

0: 384x640 1 ball, 16 players, 2 referees, 357.4ms
Speed: 1.4ms preprocess, 357.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 5

0: 384x640 15 players, 2 referees, 352.3ms
Speed: 1.9ms preprocess, 352.3ms inference, 0.5ms po